In [2]:
import pandas as pd
import numpy as np

In [40]:
# Read in data from crew csv
crew = pd.read_csv('/Users/zachwayne/Dropbox (MIT)/Netflix - Seize the Data/title.crew.tsv', sep='\t')
ratings = pd.read_csv('/Users/zachwayne/Dropbox (MIT)/Netflix - Seize the Data/title.ratings.tsv', sep='\t')
titles = pd.read_csv('/Users/zachwayne/Dropbox (MIT)/Netflix - Seize the Data/title.basics.tsv', sep='\t')


/var/folders/b9/1w_dn_kj5zgffq25ztcmwj5m0000gn/T/ipykernel_7007/1931529237.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  titles = pd.read_csv('/Users/zachwayne/Dropbox (MIT)/Netflix - Seize the Data/title.basics.tsv', sep='\t')


In [41]:
titles.replace('\\N', np.nan, inplace=True)
titles['startYear'] = titles['startYear'].astype(float)

In [6]:
titles.tail()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
9273127,tt9916848,tvEpisode,Episode #3.17,Episode #3.17,0,2010.0,NaN,NaN,"Action,Drama,Family"
9273128,tt9916850,tvEpisode,Episode #3.19,Episode #3.19,0,2010.0,NaN,NaN,"Action,Drama,Family"
9273129,tt9916852,tvEpisode,Episode #3.20,Episode #3.20,0,2010.0,NaN,NaN,"Action,Drama,Family"
9273130,tt9916856,short,The Wind,The Wind,0,2015.0,NaN,27,Short
9273131,tt9916880,tvEpisode,Horrid Henry Knows It All,Horrid Henry Knows It All,0,2014.0,NaN,10,"Adventure,Animation,Comedy"


In [42]:
# so we don't predict on the popularity of titles using the titles popularity
between_1950_2020_titles = titles[(titles['startYear']>=1950) & (titles['startYear']<=2020) & (titles['titleType'] != 'tvEpisode')]['tconst']

In [43]:
votes_and_titles = pd.merge(between_1950_2020_titles, ratings, on='tconst')


In [9]:
#Turn the directors column into a list
crew['directors'] = crew['directors'].str.split(',')

In [10]:
# explode directors column
directors = crew[['tconst', 'directors']].explode('directors')


In [44]:
# Merge the directors with the titles
directors_and_titles = pd.merge(directors, votes_and_titles, on='tconst')

In [45]:
directors_and_titles.replace('\\N', np.nan, inplace=True)

In [12]:
from sklearn.preprocessing import normalize

In [46]:
#Create 3 columns with votes, normalized votes, and rankings
director_votes = directors_and_titles.groupby('directors')['numVotes'].sum()
normalized_director_votes = normalize(director_votes.values.reshape(1, -1))
ranked_director_votes = director_votes.rank(ascending=False)
final_director_rankings = pd.DataFrame({'director':director_votes.index, 'rank':ranked_director_votes.values, 'total_votes':director_votes.values, 'normalized_votes': normalized_director_votes[0]})
final_director_rankings.sort_values('rank', inplace=True)
#final_director_rankings.to_csv('../data/interim/director_rankings.csv', index=False)

In [14]:
names = pd.read_csv('/Users/zachwayne/Dropbox (MIT)/Netflix - Seize the Data/name.basics.tsv', sep='\t')

In [47]:
# merge names with director rankings
final_director_rankings = pd.merge(final_director_rankings, names, left_on='director', right_on='nconst', how='left')
final_director_rankings.drop(columns=['nconst', 'birthYear', 'deathYear', 'primaryProfession', 'knownForTitles'], inplace=True)

In [16]:
final_director_rankings.head()

,director,rank,total_votes,normalized_votes,primaryName
0,nm0634240,1.0,13968320,0.188562,Christopher Nolan
1,nm0000229,2.0,13347289,0.180179,Steven Spielberg
2,nm0000233,3.0,11039425,0.149024,Quentin Tarantino
3,nm0000217,4.0,9615982,0.129809,Martin Scorsese
4,nm0000399,5.0,9040967,0.122047,David Fincher


Now Repeat with the Writers

In [17]:
crew['writers'] = crew['writers'].str.split(',')
writers = crew[['tconst', 'writers']].explode('writers')

In [48]:
writers_and_titles = pd.merge(writers, votes_and_titles, on='tconst')
writers_and_titles.replace('\\N', np.nan, inplace=True)

In [49]:
writer_votes = writers_and_titles.groupby('writers')['numVotes'].sum()
normalized_writer_votes = normalize(writer_votes.values.reshape(1, -1))
ranked_writer_votes = writer_votes.rank(ascending=False)
final_writer_rankings = pd.DataFrame({'writer':writer_votes.index, 'rank':ranked_writer_votes.values, 'total_votes':writer_votes.values, 'normalized_votes': normalized_writer_votes[0]})
#final_writer_rankings.to_csv('../data/interim/writer_rankings.csv', index=False)
final_writer_rankings.sort_values('rank', inplace=True)


In [50]:
# merge names with director rankings
final_writer_rankings = pd.merge(final_writer_rankings, names, left_on='writer', right_on='nconst', how='left')
final_writer_rankings.drop(columns=['nconst', 'birthYear', 'deathYear', 'primaryProfession', 'knownForTitles'], inplace=True)

In [21]:
final_writer_rankings.head()

,writer,rank,total_votes,normalized_votes,primaryName
0,nm0498278,1.0,20091111,0.245264,Stan Lee
1,nm0456158,2.0,18559908,0.226572,Jack Kirby
2,nm0634240,3.0,14453251,0.176440,Christopher Nolan
3,\N,4.0,12945663,0.158036,NaN
4,nm0000184,5.0,12760963,0.155781,George Lucas


Now for actors

In [22]:
principals = pd.read_csv('/Users/zachwayne/Dropbox (MIT)/Netflix - Seize the Data/title.principals.tsv', sep='\t')

In [23]:
# only keep actor categories
principals = principals[principals['category'].isin(['actor', 'actress', 'self'])]

In [51]:
actors_and_titles = pd.merge(principals, votes_and_titles, on='tconst')
actors_and_titles.replace('\\N', np.nan, inplace=True)

In [52]:
actors_votes = actors_and_titles.groupby('nconst')['numVotes'].sum()
normalized_actor_votes = normalize(actors_votes.values.reshape(1, -1))
ranked_actor_votes = actors_votes.rank(ascending=False)
final_actor_rankings = pd.DataFrame({'actor':actors_votes.index, 'rank':ranked_actor_votes.values, 'total_votes':actors_votes.values, 'normalized_votes': normalized_actor_votes[0]})
#final_writer_rankings.to_csv('../data/interim/writer_rankings.csv', index=False)
final_actor_rankings.sort_values('rank', inplace=True)


In [53]:
# merge names with director rankings
final_actor_rankings = pd.merge(final_actor_rankings, names, left_on='actor', right_on='nconst', how='left')
final_actor_rankings.drop(columns=['nconst', 'birthYear', 'deathYear', 'primaryProfession', 'knownForTitles'], inplace=True)

In [27]:
final_actor_rankings.head(20)

,actor,rank,total_votes,normalized_votes,primaryName
0,nm0000093,1.0,16530792,0.160947,Brad Pitt
1,nm0000158,2.0,15423302,0.150164,Tom Hanks
2,nm0000138,3.0,15226225,0.148245,Leonardo DiCaprio
3,nm0000134,4.0,14291725,0.139147,Robert De Niro
4,nm0000168,5.0,13073220,0.127283,Samuel L. Jackson
5,nm0000288,6.0,12795612,0.124581,Christian Bale
6,nm0000375,7.0,12435752,0.121077,Robert Downey Jr.
7,nm0000246,8.0,12215948,0.118937,Bruce Willis
8,nm0000354,9.0,11687925,0.113796,Matt Damon
9,nm0000151,10.0,11444477,0.111426,Morgan Freeman


In [54]:
# write all rankings to csv
final_director_rankings.to_csv('/Users/zachwayne/Dropbox (MIT)/Netflix - Seize the Data/director_rankings.csv', index=False)
final_writer_rankings.to_csv('/Users/zachwayne/Dropbox (MIT)/Netflix - Seize the Data/writer_rankings.csv', index=False)
final_actor_rankings.to_csv('/Users/zachwayne/Dropbox (MIT)/Netflix - Seize the Data/actor_rankings.csv', index=False)